In [8]:
pip install psycopg2-binary

   ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import arcpy
import psycopg2


In [3]:
#create list of coordinates

coordinates = [ (0,0), (10,0), (10,10), (5,15), (0,10)]

points = [arcpy.Point(*coords) for coords in coordinates]

In [4]:
#create array from the points
array = arcpy.Array(points)

#create polygon from array
polygon = arcpy.Polygon(array)

print(polygon)

<geoprocessing describe geometry object object at 0x000001A0A55931E0>


In [5]:
# Convert my polygon to WKT
wkt = polygon.WKT
print(wkt)

MULTIPOLYGON (((0 0, 10 0, 10 10, 5 15, 0 10, 0 0)))


In [16]:
connection = psycopg2.connect(
    dbname="lab0",
    user="postgres",
    password='S_FQm^"tjA)kbFN@',
    host="34.45.180.162",
    port="5432"
)

cursor = connection.cursor()

# Check if the table exists and create it if it doesn't
create_table_query = """
CREATE TABLE IF NOT EXISTS my_polygon (
    id SERIAL PRIMARY KEY,
    geom GEOMETRY
);
"""
cursor.execute(create_table_query)

# Insert the polygon WKT into the table
insert_query = """
INSERT INTO my_polygon (geom)
VALUES (ST_GeomFromText(%s, 4326))
"""
cursor.execute(insert_query, (wkt,))
connection.commit()

cursor.close()
connection.close()